# Trading EDA

To install conda and binance packages to this notebook uncomment the code below

In [1]:
#%conda install -c plotly plotly=5.9.0
#%conda install pip
#%conda install twisted
%pip install plotly==5.9.0
%pip install twisted
%pip install binance-connector==1.13.0
%pip install pandas
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

/home/dima/anaconda3/lib/python3.7/site-packages/cryptography/hazmat/backends/openssl/x509.py:17: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Init binance client

In [2]:
from binance.spot import Spot as Client
import os
import sys
import yaml

# Load 
def load_config():
    """
    Load config from cfg folder respecting the order: defaults, app.yaml, environment vars
    """
    # Defaults
    cfg_dir="./../biml/cfg"
    default_cfg_path = f"{cfg_dir}/app-defaults.yaml"
    with open(default_cfg_path, "r") as appdefaults:
        config = yaml.safe_load(appdefaults)

    # Custom config, should contain custom information,
    cfg_path = f"{cfg_dir}/app.yaml"
    if os.path.exists(cfg_path):
        with open(cfg_path) as app:
            config.update(yaml.safe_load(app))
    else:
        sys.exit(
            f"Config {cfg_path} not found. Please copy cfg/app-defaults.yaml to {cfg_path} "
            f"and update connection info there.")

    # Enviroment variabless
    config.update(os.environ)
    return config


# Create binance client
config=load_config()
key,secret,url=config["biml.connector.key"],config["biml.connector.secret"],config["biml.connector.url"]
client = Client(key=key, secret=secret, base_url=url, timeout=10)


Get account info

In [3]:
account=client.account()
print(account)


{'makerCommission': 0, 'takerCommission': 0, 'buyerCommission': 0, 'sellerCommission': 0, 'commissionRates': {'maker': '0.00000000', 'taker': '0.00000000', 'buyer': '0.00000000', 'seller': '0.00000000'}, 'canTrade': True, 'canWithdraw': False, 'canDeposit': False, 'brokered': False, 'requireSelfTradePrevention': False, 'updateTime': 1678006474349, 'accountType': 'SPOT', 'balances': [{'asset': 'BNB', 'free': '1000.00000000', 'locked': '0.00000000'}, {'asset': 'BTC', 'free': '0.99900000', 'locked': '0.00100000'}, {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'}, {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'}, {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'}, {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'}, {'asset': 'USDT', 'free': '10000.00653000', 'locked': '0.00000000'}, {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}], 'permissions': ['SPOT']}


Get trade bot orders from sqlite db

In [4]:
def get_trades():
    import sqlite3
    db_path="../data/biml.db"

    with sqlite3.connect(db_path) as conn:
        trades=pd.read_sql_query("SELECT * FROM trade ORDER BY open_time",conn,
                                 parse_dates=["open_time", "close_time"])#.set_index("open_time", drop=False)
    trades["profit"] = trades["close_price"]-trades["open_price"]
    return trades

trades=get_trades()
trades.tail()

,id,ticker,side,open_time,open_price,open_order_id,stop_loss_price,stop_loss_order_id,trailing_delta,quantity,close_time,close_price,close_order_id,profit
7,8,BTCUSDT,BUY,2023-03-05 08:38:25.894413,22365.79,2859145,22343.73,2859151,None,0.001,2023-03-05 11:39:05.250116,22348.43,2859550,-17.36
8,9,BTCUSDT,BUY,2023-03-05 08:39:15.329011,22345.24,2859639,22319.51,2859651,None,0.001,2023-03-05 11:40:05.903918,22350.34,2860107,5.10
9,10,BTCUSDT,BUY,2023-03-05 08:40:16.595495,22348.79,2860178,22327.78,2860179,None,0.001,2023-03-05 11:43:14.337015,22349.32,2861545,0.53
10,11,BTCUSDT,BUY,2023-03-05 08:43:24.687356,22351.04,2861606,22328.18,2861608,None,0.001,2023-03-05 11:54:24.411336,22340.66,2866218,-10.38
11,12,BTCUSDT,BUY,2023-03-05 08:54:34.493511,22337.28,2866274,22318.32,2866281,None,0.001,NaT,NaN,None,NaN


Get opened trades

In [5]:
trades[trades.close_time.isna()].tail()

,id,ticker,side,open_time,open_price,open_order_id,stop_loss_price,stop_loss_order_id,trailing_delta,quantity,close_time,close_price,close_order_id,profit
11,12,BTCUSDT,BUY,2023-03-05 08:54:34.493511,22337.28,2866274,22318.32,2866281,None,0.001,NaT,NaN,None,NaN


In [6]:
# # Get orders
# import pandas as pd
# import numpy as np
# symbol="BTCUSDT"

# # Get orders, but market orders have price=0
# orders=pd.DataFrame(client.get_orders(symbol))
# if not orders.empty: orders=orders.astype({'time': 'datetime64[ms]','updateTime':'datetime64[ms]','price':'float64','origQty':'float64'}).set_index('time',drop=False)

# # Get trades and pull orders' prices from there
# trades=pd.DataFrame( client.my_trades(symbol))
# if not trades.empty: trades=trades.astype({'time': 'datetime64[ms]', 'price':'float64'}).set_index('time',drop=False)    
# orders=orders.drop("price",axis=1).merge(trades[["orderId","price"]],on="orderId")

# # Calculate profit of closing orders
# orders["delta_price"]=orders["price"].diff()
# orders["profit"] = orders["delta_price"]*orders["origQty"]
# orders.loc[~orders["type"].isin(["LIMIT_MAKER", "STOP_LOSS_LIMIT"]),["profit"]]=np.nan

# orders["time"] = orders["updateTime"]
# orders.set_index("time", drop=False, inplace=True)
# orders.tail()


Look at opened stop loss and take profit orders

In [7]:
symbol="BTCUSDT"
orders_opened=pd.DataFrame(client.get_open_orders(symbol))
orders_opened.tail()

,symbol,orderId,orderListId,clientOrderId,price,origQty,executedQty,cummulativeQuoteQty,status,timeInForce,type,side,stopPrice,trailingDelta,trailingTime,icebergQty,time,updateTime,isWorking,workingTime,origQuoteOrderQty,selfTradePreventionMode
0,BTCUSDT,2866281,-1,SXhFge7z9wJ6mbrMKtoD46,22295.98000000,0.00100000,0.00000000,0.00000000,NEW,GTC,STOP_LOSS_LIMIT,SELL,22318.32000000,200,1678007078872,0.00000000,1678006474349,1678007078872,False,-1,0.00000000,NONE


Get candles from binance

In [15]:
def get_last_candles():
    candle_columns = ["open_time", "open", "high", "low", "close", "vol", "close_time", "quote_asset_volume",
                      "number_of_trades", " taker_buy_base_asset_volume", "taker_buy_quote_asset_volume",
                      "ignore"]    
    end_date=trades.open_time.max() if not trades.empty else datetime.utcnow()
    end_time_millis=end_date.value//1000000
    raw_candles = client.klines(symbol=symbol,
                                    interval="1m",
                                    #limit=10,
                                    limit=24*60*60#,
                                    #endTime=end_time_millis
                               )
    df = pd.DataFrame(raw_candles, columns=candle_columns)
    df["open_time"] = pd.to_datetime(df["open_time"], unit='ms')
    df["close_time"] = pd.to_datetime(df["close_time"], unit='ms')
    df.set_index("open_time", drop=False, inplace=True)
        # Convert strings to float prices
    df[["open", "high", "low", "close"]] = df[["open", "high", "low", "close"]].astype(float)
    return df

def clean_candles(df):
    """ Remove outliers """
    q_low = df["low"].quantile(0.01)
    q_high  = df["high"].quantile(0.99)
    df=df[df["high"]< q_high]
    df=df[df["low"] > q_low ]
    return df

candles = get_last_candles()
candles = clean_candles(candles)
candles.tail()

,open_time,open,high,low,close,vol,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
open_time,,,,,,,,,,,,
2023-03-05 10:25:00,2023-03-05 10:25:00,22417.52,22448.15,22409.76,22438.79,7.31718800,2023-03-05 10:25:59.999,164094.79437150,164,6.09572000,136700.12438972,0
2023-03-05 10:26:00,2023-03-05 10:26:00,22440.27,22440.27,22368.35,22419.00,6.11468100,2023-03-05 10:26:59.999,137078.14415730,128,3.26128000,73131.06512364,0
2023-03-05 10:27:00,2023-03-05 10:27:00,22419.30,22423.81,22367.47,22409.70,6.78792600,2023-03-05 10:27:59.999,152112.13709538,110,4.38671000,98335.79578023,0
2023-03-05 10:28:00,2023-03-05 10:28:00,22409.90,22421.84,22408.77,22409.04,5.41781100,2023-03-05 10:28:59.999,121435.32482857,101,3.54687300,79502.09438635,0
2023-03-05 10:29:00,2023-03-05 10:29:00,22409.30,22409.55,22395.42,22396.13,3.53709600,2023-03-05 10:29:59.999,79242.84984334,68,1.11569600,24994.69816879,0


In [9]:
candles.tail()

,open_time,open,high,low,close,vol,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
open_time,,,,,,,,,,,,
2023-03-05 08:50:00,2023-03-05 08:50:00,22343.35,22343.58,22336.78,22338.37,6.23906900,2023-03-05 08:50:59.999,139388.25756800,108,1.51273100,33795.14698246,0
2023-03-05 08:51:00,2023-03-05 08:51:00,22338.37,22346.89,22338.37,22341.57,5.16288400,2023-03-05 08:51:59.999,115354.99095671,96,3.53609200,79005.04126032,0
2023-03-05 08:52:00,2023-03-05 08:52:00,22341.16,22344.55,22337.67,22343.76,4.43400100,2023-03-05 08:52:59.999,99056.87323235,87,2.83331000,63296.64666989,0
2023-03-05 08:53:00,2023-03-05 08:53:00,22344.05,22344.35,22343.01,22343.31,3.32756300,2023-03-05 08:53:59.999,74350.34668015,58,2.16166100,48299.81880547,0
2023-03-05 08:54:00,2023-03-05 08:54:00,22343.31,22343.31,22337.28,22337.70,4.55574000,2023-03-05 08:54:59.999,101771.24453380,86,2.43720500,54443.77607057,0


## EDA

### Look at trades quantity cumulative sum

## Look at trades on candlestick chart

In [16]:
# Install a conda package in the current Jupyter kernel
#%conda install -c plotly plotly=5.9.0
# Draw candlechart
import plotly as py
from plotly import graph_objects as go
import pandas as pd
import numpy as np
import datetime

df=candles.copy()
#df=clean_candles(df)
fig = go.Figure(data=[ \
                    go.Candlestick(\
                        x=df.index,\
                        open=df['open'],\
                        high=df['high'],\
                        low=df['low'],\
                        close=df['close'])
                     ])

# Customize ay to distribute captions by y so they do not overlap by x
ay,aymax,aystep=0,50,10
#start_time=df.open_time.min()
# Annotate buy, sell on the chart
for i,trade in trades[(~trades.open_time.isna()) & (trades.open_time > df.open_time.min())].iterrows():
    ay=(ay+aystep)%aymax
    color='green' if trade['side']=="BUY" else 'red'

    # Draw open order
    open_text='buy' if trade['side']=="BUY"  else 'sell'
    open_hover=f"{trade['open_time']} {open_text} at {str(trade['open_price'])}"
    fig.add_annotation(x=trade['open_time'], \
                       y=trade['open_price'],\
                       text=open_text,\
                       hovertext=open_hover,\
                       showarrow=True,arrowhead=5,arrowcolor=color,ay=-ay,arrowwidth=2,
                       font={'color':color}
                      )
for i,trade in trades[(~trades.close_time.isna()) & (trades.close_time > df.close_time.min())].iterrows():

    print("close_time:"+str(trade.close_time.value))
    # Draw close order
    #profit = trade["close_price"]-trade["open_price"] if trade["close_price"] else None
    status= f", profit: {round(trade.profit, 10)}" if trade.profit else ""
    close_text='close buy' if trade['side']=="BUY"  else 'close sell'
    close_hover=f"{trade['close_time']} {close_text} at {str(trade['close_price'])}{status}"
    fig.add_annotation(x=trade['close_time'], \
                       y=trade['close_price'],\
                       text=close_text,\
                       hovertext=close_hover,\
                       showarrow=True,arrowhead=5,arrowcolor=color,ay=-ay,arrowwidth=2,
                       font={'color':color}
                      )

fig.show()


close_time:1678015736222758000
close_time:1678015864110326000
close_time:1678016009058266000
close_time:1678016219500202000
close_time:1678016260410161000
close_time:1678016295368447000
close_time:1678016345250116000
close_time:1678016405903918000
close_time:1678016594337015000
close_time:1678017264411336000


## Profit

In [11]:
#import matplotlib.pyplot as plt
import plotly.express as px
# Calc profit, considering only closing trades
profit=trades.copy()
profit["cum_profit"] = profit["profit"].cumsum()
profit["color"]=np.where(profit.profit >=0, "green","red")
profit.tail()

#Plot
fig = px.line(profit, x="open_time", y="profit")
fig.show()

In [12]:
import matplotlib.pyplot as plt
import plotly.express as px
# Calc profit, considering only closing trades
profit=trades[~trades["profit"].isna()].copy()
#profit["profit"]=profit["price"].diff()*(profit["qty"]/profit["qty"])
profit["cum_profit"] = profit["profit"].cumsum()

#Plot
fig = px.line(profit, y="cum_profit")
fig.show()

## Predicted and actual low/high

In [13]:
import os
import glob
import pandas as pd
from datetime import timedelta,date
import matplotlib.pyplot as plt
import plotly.express as px

def read_predictions(start_date_inc, end_date_inc):
    """ Read days from start to end from local predictions history"""
    symbol="BTCUSDT"    
    data_dir=f"./../model-dev/PredictLowHighCandlesStrategy/Xy"
    print(f"start date:{start_date_inc}, end date:{end_date_inc}, all inclusive")
    df = pd.DataFrame()
    candles_list=[]
    for n in range(int ((end_date_inc - start_date_inc).days)+1):
        cur_date = start_date_inc + timedelta(n)    
        candles_path=f"{data_dir}/{cur_date}_{symbol}_candles.csv"     
        if not os.path.exists(candles_path): continue
        #print(f"Read {cur_date} candles from {csv_path}")
        cur_candles=pd.read_csv(candles_path,index_col="close_time")
        candles_list.append(cur_candles)
    df = pd.concat(candles_list)
    return df


# Read candles for last 3 days
last_date=trades.open_time.max().date()
first_date=last_date - timedelta(days=0)

df = read_predictions(first_date, last_date)
px.line(df[["low","high","fut_low","fut_high"]], markers=True)


start date:2023-03-05, end date:2023-03-05, all inclusive


## Look at the account

In [14]:
print(account)

{'makerCommission': 0, 'takerCommission': 0, 'buyerCommission': 0, 'sellerCommission': 0, 'commissionRates': {'maker': '0.00000000', 'taker': '0.00000000', 'buyer': '0.00000000', 'seller': '0.00000000'}, 'canTrade': True, 'canWithdraw': False, 'canDeposit': False, 'brokered': False, 'requireSelfTradePrevention': False, 'updateTime': 1678006474349, 'accountType': 'SPOT', 'balances': [{'asset': 'BNB', 'free': '1000.00000000', 'locked': '0.00000000'}, {'asset': 'BTC', 'free': '0.99900000', 'locked': '0.00100000'}, {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'}, {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'}, {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'}, {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'}, {'asset': 'USDT', 'free': '10000.00653000', 'locked': '0.00000000'}, {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}], 'permissions': ['SPOT']}
